train.csv: A full training dataset with the following attributes:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks the article as potentially unreliable

---


1: unreliable

0: reliable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re # this is use to "search text" in a document
from nltk.corpus import stopwords # nltk --> natrual languages toolkit / text we are going to deal with 
# corpus --> body of the particular text/ import content of the text
# stopwords --> those words which doesn't add much value to a paragraph (Artices, Wh)
from nltk.stem.porter import PorterStemmer # stemming takes a word and removes prefix and suffix of the word and return the root word of it
from sklearn.feature_extraction.text import TfidfVectorizer # convert text into --> feature vectors (numbers)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# downloading nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [ ]:
file_path = "/content/drive/MyDrive/Machine Learning (Siddhardhan)/ML Projects/Project 4. Fake News Prediction/train.csv"

In [ ]:
news_df = pd.read_csv(file_path)

In [ ]:
news_df.shape

(20800, 5)

In [ ]:
news_df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# replacing null value with empty string
news_df = news_df.fillna("")

Merging the "author name" and "news title"

In [ ]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [ ]:
print(news_df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
# Seperating the data & label

X = news_df.drop(columns='label', axis = 1)
Y = news_df['label']

In [ ]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [ ]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Stemming:


---
Stemming is the process of reducing a word to its Root word

example: actor, actress, acting ---> act


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content): # content --> is input.
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) # [^a-zA-Z] --> alphabet chara sob remove --->number, commas, quotes will be replace by space---> content er
  stemmed_content = stemmed_content.lower() # converted into lower alphabet
  stemmed_content = stemmed_content.split() # words and text will be split and converted into list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # word gulo ke stem kore "root word" e anteste--> word astese "stemmed content" theke zodi na word "stopwords" na hoy
  stemmed_content = ' '.join(stemmed_content) # sob word gulo join kora hoyeche
  return stemmed_content

In [ ]:
# apply the function into "Content column"

news_df['content'] = news_df['content'].apply(stemming)

In [ ]:
print(news_df['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


Separating the data and label


In [ ]:
X = news_df['content'].values
Y = news_df['label'].values

In [ ]:
X

array(['darrel lucu hou dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exerci balkan',
       'david swanson keep f aliv'], dtype=object)

In [ ]:
Y

array([1, 0, 1, ..., 0, 1, 1])

Converting the Textual data to Numerical data

In [ ]:
vectorizer = TfidfVectorizer() # Tf --> Term Frequency and idf --> inverse document frequency
# basically counts the number of times a particular word is repeating in a document so in a particular text, paragraph
# repeatating tells that " its a important word" and assigns particular numerical value to that word

# idf --> sometimes a word which is repeated several times doesn't have meaning in it
# Example --> Avengers movie review --> that will be contains avengers name in it more but it is not useful
# idf --> finds those values which are repeating so many times and it detecs that those words are not significant and it reduces its importance value

# by this feature vector creates which is --> number

X = vectorizer.fit_transform(X)

#X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 15553)	0.28485063562728646
  (0, 2464)	0.3676519686797209
  (0, 7612)	0.24785219520671603
  (0, 8546)	0.29212514087043684
  (0, 2933)	0.2468450128533713
  (0, 13348)	0.2565896679337957
  (0, 4917)	0.233316966909351
  (0, 263)	0.27010124977708766
  (0, 3759)	0.2705332480845492
  (0, 6930)	0.21874169089359144
  (0, 8822)	0.3635963806326075
  (0, 3567)	0.3598939188262559
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 16656)	0.3025156488372128
  (1, 1481)	0.2957471154505952
  (1, 2790)	0.19208753385709676
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  (2, 15478)	0.41544962664721613
  (2, 2917)	0.3179886800654691
  :	:
  (20797, 1274)	0.3355398041847889
  (20797, 12998)	0.24837050368318928
  (20797, 12227)	0.2727640214571724
  (20797, 14838)	0.311742474262946
  (20797, 12022)	0.2479002225274413
  (20797, 9431)	0.293949478

Splitting the dataset to training and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

Training the mode: Logistic Regression

In [ ]:
lo_model = LogisticRegression()

In [ ]:
lo_model.fit(X_train, Y_train) # plot sigmoid curve

LogisticRegression()

Evaluation

---
Accuracy Score


In [ ]:
training_data_prediction = lo_model.predict(X_train)

In [ ]:
training_data_accuracy = accuracy_score(Y_train,training_data_prediction)

In [ ]:
print("Accuracy score of the traning data: ", training_data_accuracy)

Accuracy score of the traning data:  0.9866586538461538


Accuracy on the test data will tell how goog our model is performing

In [ ]:
testing_data_prediction = lo_model.predict(X_test)

In [ ]:
testing_data_accuracy = accuracy_score(Y_test,testing_data_prediction)

In [ ]:
print("Accuracy score of the testing data: ", testing_data_accuracy)

Accuracy score of the testing data:  0.9790865384615385


Predicted System

In [ ]:
X_new = X_test[696]

prediction = lo_model.predict(X_new)

print(prediction)

if(prediction[0]==0):
  print("Valid News")
else:
  print("Fake News")


[0]
Valid News


In [ ]:
print(Y_test[696])

0
